In [ ]:
lr = 1e-6
dataset = 'manual'
max_train_steps = 2000

# Installation

In [ ]:
checkpoints_path = 'gdrive/MyDrive/inpainting/checkpoints'
train_data_path = f'gdrive/MyDrive/inpainting/train_data_{dataset}'
# class_data only for prior preservation loss
class_data_path = f'gdrive/MyDrive/inpainting/class_data'
synthetic_data_path = 'gdrive/MyDrive/inpainting/synthetic_data/manual'

In [ ]:
from google.colab import drive
drive.mount('gdrive/')

In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!git clone https://github.com/huggingface/diffusers
!pip install -U -r diffusers/examples/research_projects/dreambooth_inpaint/requirements.txt
!pip install accelerate
!pip install bitsandbytes
!pip install xformers

In [ ]:
!accelerate config default

In [ ]:
!pip install -U torch==2.0.0+cu118 torchvision==0.15.1+cu118 --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install -U diffusers

In [ ]:
from diffusers import StableDiffusionInpaintPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
import torch
import PIL
from datetime import datetime

# Training

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --instance_data_dir=$train_data_path \
  --output_dir=$checkpoints_path \
  --instance_prompt="satellite imagery, wildfire, landsat-8" \
  --resolution=256 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --gradient_checkpointing \
  --use_8bit_adam \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=3000

# Training with prior preservation loss

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --instance_data_dir=$train_data_path \
  --class_data_dir=$class_data_path \
  --output_dir=$checkpoints_path \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --instance_prompt="thirty meter resolution satellite image, localized wildfire in a broad environment" \
  --class_prompt="thirty meter resolution satellite image, localized wildfire in a broad environment" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --use_8bit_adam \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=1000

# Training text encoder

In [ ]:
!accelerate launch diffusers/examples/research_projects/dreambooth_inpaint/train_dreambooth_inpaint.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-inpainting"  \
  --train_text_encoder \
  --instance_data_dir=$train_data_path \
  --output_dir=$checkpoints_path \
  --instance_prompt="satellite imagery, wildfire, landsat-8" \
  --resolution=256 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=2 \
  --learning_rate=$lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=$max_train_steps \
  --checkpointing_steps=3000